In [3]:
!pip install kafka-python

In [4]:
from kafka import KafkaConsumer
from json import loads
import time
from pyspark.sql import SparkSession
from multiprocessing import Process, Manager
import json

In [5]:
columns = ["vendor_name","Trip_Pickup_DateTime","Trip_Dropoff_DateTime",
           "Passenger_Count","Trip_Distance","Start_Lon","Start_Lat",
           "Rate_Code","store_and_forward","End_Lon","End_Lat","Payment_Type",
           "Fare_Amt","surcharge","mta_tax","Tip_Amt","Tolls_Amt","Total_Amt"]

In [6]:
consumer = KafkaConsumer('ny_tripdata',
                         bootstrap_servers=['docker-compose_kafka_1:29092'],
                         auto_offset_reset='earliest',
                         group_id='group')

In [7]:
manager = Manager()
messages = manager.Queue()

In [8]:
messages

<AutoProxy[Queue] object, typeid 'Queue' at 0x7fcf90f0a220>

In [10]:
def consumeData(consumer, messages):
    try:
        for message in consumer:
            msg = json.loads(message.value.decode("utf-8"))
            messages.put(msg)
    except Exception as e:
        print(e)

t1 = Process(target=consumeData, args=(consumer, messages, ))

In [11]:
t1.start()

In [19]:
messages.get()

{'vendor_name': 'CMT',
 'Trip_Pickup_DateTime': '2009-01-05 05:29:14',
 'Trip_Dropoff_DateTime': '2009-01-05 05:45:09',
 'Passenger_Count': '1',
 'Trip_Distance': '7.7999999999999998',
 'Start_Lon': '-73.995064999999997',
 'Start_Lat': '40.688806999999997',
 'Rate_Code': None,
 'store_and_forward': None,
 'End_Lon': '-73.974406000000002',
 'End_Lat': '40.759920999999999',
 'Payment_Type': 'Cash',
 'Fare_Amt': '20.199999999999999',
 'surcharge': '0',
 'mta_tax': None,
 'Tip_Amt': '0',
 'Tolls_Amt': '0',
 'Total_Amt': '20.199999999999999'}

In [ ]:
t1.terminate()
del t1

In [13]:
spark = SparkSession \
    .builder \
    .appName("NY_tripdate") \
    .config("spark.jars", "/home/jovyan/work/postgresql-42.2.23.jar") \
    .getOrCreate()

21/07/21 19:48:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [21]:
# df = spark.read \
#     .option("dbtable", "person") \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql://dockerfiles_db_1:5432/postgres") \
#     .option("user", "postgres").option("password", "postgres1234") \
#     .option("driver", "org.postgresql.Driver") \
#     .load()

# df.printSchema()

In [23]:
# df.show()

In [19]:
# spark.write \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql://dockerfiles_db_1:5432/postgres") \
#     .option("user", "postgres").option("password", "postgres1234") \
#     .option("driver", "org.postgresql.Driver") \
#     .option("createTableColumnTypes", "name CHAR(64), comments VARCHAR(1024)")
# #     .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
# #           properties={"user": "username", "password": "password"})
